In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import seaborn.objects as so

In [2]:
directory_path_box_max = r'../../models/Box/3/max'
directory_path_box_min = r'../../models/Box/3/min'

In [7]:
directory_path_min = r'../../../models/KMeans/output/q3_c3/q3_c3_top_10_normal2024-04-30 04:57:39.parquet'
directory_path_max = r'../../../models/KMeans/output/q3_c3/q3_c3_top_10_normal2024-04-30 04:57:39.parquet'
df_20_col = pd.read_parquet('../../../models/KMeans/output/seatunnal_20col.parquet')
df_original = pd.read_parquet('../../../Sonar/seatunnel_all_information.parquet')

In [8]:
df_min = pd.read_parquet(directory_path_min)
df_max = pd.read_parquet(directory_path_max)

In [9]:
df_min.columns

เรียงลำดับมากไปน้อย q3

In [10]:
df_max_sort = df_max[
    ['col', 'cluter0_q3', 'cluter1_q3', 'cluter2_q3', 'cluter0_q1', 'cluter1_q1', 'cluter2_q1', 'label']]

In [11]:
df_max_sort

loop sort values of q3 minimux

In [13]:
q3 = df_max_sort.iloc[:, [1, 2, 3]]
q3

In [ ]:
q1 = df_max_sort.iloc[:, [4, 5, 6]]

In [ ]:
def loop_check_value_q(df):
    index_list = []
    min_values_list = []
    max_values_list = []
    for i in range(len(df)):
        x = df.iloc[i].sort_values()
        print('index:', df.index[i])
        # print('value:', x)
        min = x[0]
        max = x[2]
        print('min:', x[0])
        print('max:', x[2])
        index_list.append(df.index[i])
        min_values_list.append(min)
        max_values_list.append(max)
    
    return index_list, min_values_list, max_values_list

In [ ]:
index, min_q3 , max_q3 = loop_check_value_q(q3)

In [ ]:
min_q3

In [ ]:
max_q3

min and max value of q1

In [ ]:
index, min_q1 , max_q1 = loop_check_value_q(q1)

min and max value of q3

In [ ]:
concat_col = pd.concat([pd.DataFrame(index, columns=['index']), pd.DataFrame(min_q1, columns=['min']), pd.DataFrame(max_q3, columns=['max'])], axis=1) 

In [ ]:
concat_col = pd.DataFrame(concat_col)
concat_col.set_index('index', inplace=True)
concat_col

In [ ]:
df_max_sort

In [ ]:
df_max_class = pd.merge(df_max_sort, concat_col, left_index=True, right_index=True)

In [ ]:
df_max_class

In [ ]:
df_max_class.to_parquet('../../models/KMeans/output/q3_c3/df_max_time_class.parquet')

Model classification of q3 SVM

In [ ]:
for i in df_max_class.iloc:
    # print("I ::", i)
    
    df_compare = df_20_col[list(i['col'])]
    # print("DF ::", df_compare)
    
    arr = np.array(i['label'])
    # print("Arr ::", arr)
    
    df_arr = pd.DataFrame(arr.reshape(-1,1), columns=['label'])
    # print("DF Arr ::", df_arr)
    
    df_col_combined = pd.concat([df_compare, df_arr], axis=1)
    df_col_combined = pd.concat([df_col_combined, df_original['total_time']], axis=1)
    
    date = pd.to_timedelta(df_col_combined['total_time'])
    time_hours = date.dt.total_seconds() / 3600

    df_col_combined['hours'] = time_hours
    df_col_combined['min'] = [i['min']] * len(df_col_combined)
    df_col_combined['max'] = [i['max']] * len(df_col_combined)
    
    print("DF Combined ::", df_col_combined)